# CMB-S4: DRAFT tool

## loading necessary python modules

In [1]:
%load_ext autoreload
%autoreload 2

#%pylab notebook
%matplotlib inline
from pylab import *
import os

import argparse, sys, numpy as np, scipy as sc, warnings, os
sys.path.append('modules/')
import tools

#import matplotlib.cbook
warnings.filterwarnings('ignore',category=RuntimeWarning)
#warnings.filterwarnings('ignore', category=DeprecationWarning)
#warnings.filterwarnings('ignore', category=matplotlib.cbook.mplDeprecation)

In [2]:
fsky = 0.57
fix_params = ['mnu']
prior_dict = {'tau': 0.007}
results_folder = 'results/'
ilc_pgm_name = 'get_ilc_weights_and_residuals.py'
fisher_pgm_name = 'get_fisher_withdelensing.py'
ilc_opfname = '%s/s4_cmb_ilc.npy' %(results_folder)
fisher_opfname = '%s/s4_cmb_fisher.npy' %(results_folder)

In [3]:
#execute
cmd1 = 'python3 %s -opfname %s' %(ilc_pgm_name, ilc_opfname)
print('\n######')
print(cmd1)
os.system(cmd1)
cmd2 = 'python3 %s -ilc_fname %s -opfname %s' %(fisher_pgm_name, ilc_opfname, fisher_opfname)
print('\n######')
print(cmd2)
#os.system(cmd2)
print('######')


######
python3 get_ilc_weights_and_residuals.py -opfname results//s4_cmb_ilc.npy

Start ILC calculation
	load parameter file
	get experimental specs
	get beam and noise specs
	get noise spectra nls
	get beam bls
	get the CMB, noise, and foreground covariance
	get the ILC residuals and weights
	get weighted foreground spectra
All done - check results//s4_cmb_ilc.npy for ILC residuals


######
python3 get_fisher_withdelensing.py -ilc_fname results//s4_cmb_ilc.npy -opfname results//s4_cmb_fisher.npy
######


In [4]:
#now read the Fisher matrix and get constraints
which_spectra = 'delensed'
results_dict = np.load(fisher_opfname, allow_pickle = True).item()
cosmo_params = results_dict['cosmoParams']
fisher_mat_dict = results_dict['fisherGaussian'][0]
fisher_mat = fisher_mat_dict[which_spectra] * fsky

In [5]:
#fix params and add priors
fisher_mat, cosmo_params = tools.fn_fix_params(fisher_mat, cosmo_params, fix_params)
cosmo_params = np.asarray(cosmo_params)
#add prior
fisher_mat = tools.fn_add_prior(fisher_mat, cosmo_params, prior_dict)

In [6]:
cov_mat = np.linalg.inv(fisher_mat)
cosmo_constraints = np.diag(cov_mat)**0.5

print('One sigma errors are listed below:\n')
for p, sigma in zip(cosmo_params, cosmo_constraints):
    print('\tParam = %s; %g' %(p, sigma))

One sigma errors are listed below:

	Param = omega_c_h2; 0.000601973
	Param = omega_b_h2; 3.99335e-05
	Param = N_eff; 0.0307941
	Param = A_s; 1.77282e-11
	Param = n_s; 0.00275588
	Param = tau; 0.00464958
	Param = theta_s; 6.75906e-07


SystemExit: 

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
